In [61]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def TMensual(Tmort):
    """
    Interpola los valores de l(x) en pasos de 1/12 (mensual) y calcula q(x) y p(x).

    Parameters:
    Tmort (DataFrame): DataFrame que contiene las columnas 'x' y 'l(x)'.

    Returns:
    DataFrame: DataFrame con valores interpolados de x, l(x), q(x) y p(x).
    """

    # Aseguramos que Tmort tiene las columnas adecuadas
    if 'x' not in Tmort.columns or 'l(x)' not in Tmort.columns:
        raise ValueError("El DataFrame debe contener las columnas 'x' y 'l(x)'")

    # Obtener los valores de x y l(x)
    x_values = Tmort['x'].values
    lx_values = Tmort['l(x)'].values

    # Crear una función de interpolación
    interp_function = interp1d(x_values, lx_values, kind='linear', fill_value='extrapolate')

    # Crear los nuevos valores de x con un paso de 1/12, hasta 110 (incluyendo 110)
    new_x = np.arange(x_values[0], 110 + 1/12, 1/12)  # Incrementos mensuales hasta 110

    # Calcular los valores de l(x) interpolados
    new_lx = interp_function(new_x)

    # Inicializar los arrays para q(x) y p(x)
    qx_values = np.zeros(len(new_x) - 1)  # Inicializar el array para q(x)
    px_values = np.zeros(len(new_x) - 1)  # Inicializar el array para p(x)

    # Calcular q(x) y p(x) para cada nuevo valor de x
    for i in range(len(new_x)-1):
        if new_lx[i] > 0:  # Evitar división por cero
            qx_values[i] = 1 - (new_lx[i + 1] / new_lx[i])  # q(x) = 1 - l(x + 1/12) / l(x)
        else:
            qx_values[i] = 0  # Si l(x) es 0, q(x) también debe ser 0

        px_values[i] = 1 - qx_values[i]  # p(x) = 1 - q(x)


    # Crear un nuevo DataFrame con los valores interpolados y calculados
    interpolated_df = pd.DataFrame({
        'x': new_x,  # Incluir hasta 110
        'l(x)': new_lx,  # Excluir el último por la misma razón
        'q(x)': np.append(qx_values, 1),  # Añadir el valor de q(110)
        'p(x)': np.append(px_values, 0)  # Añadir el valor de p(110)
    })

    return interpolated_df

def tmort_filtered(df,edad):
    qx_vec=df[df['x']>=edad]
    return qx_vec

In [64]:
##Finanzas - Interes Compuesto

def vf_cte(pago, n, i):

#Pago -> Valor de la mensualidad
#n -> Número de periodos
#i -> Interes de la periodicidad n

  pagos = np.ones(n) * pago
  print(pagos.sum())
  cao_factor = np.ones(n) * (1 + i) ** np.arange(n,0,-1)
  resp_vector = pagos * cao_factor
  resp = resp_vector.sum()
  return resp


In [65]:
#Datos
edad_x=30
sexo='H'
pago=0.04 # el 12% restante lo aporta el empleador
salario = 7000000
edad_jubilacion=62
tasa_anual=0.04
#Acumulado=960000

#Calcule el ahorro total de los 62 años

##Tenga en cuenta, el primer pago de la renta es justo a los 30 año, como si fuera hoy
pago_mensual=salario*0.16
n=(edad_jubilacion-edad_x)*12
i=tasa_anual/12
vf=vf_cte(pago_mensual,n,i)
print(vf)
print(n*pago_mensual)


430080000.0
872800998.1122966
430080000.0


In [73]:
print(f"\033[1mPago mensual:\033[0m {pago_mensual}")
print(f"\033[1mPeriodos:\033[0m {n}")
print(f"\033[1mInteres:\033[0m {i:.4f}")

Pago mensual: 1120000.0
Periodos: 384
Interes: 0.0033


In [71]:
ahorro_total=vf
print(f"\033[1mAhorro total:\033[0m {vf:.2f}")

Ahorro total: 872800998.11


In [74]:
Tmort_h = pd.read_excel('/content/TM_hombres.xlsx')
Tmort_m= pd.read_excel('/content/TM_mujeres.xlsx')


In [75]:
T_hombre = TMensual(Tmort_h)
T_mujer = TMensual(Tmort_m)


In [77]:
T_hombre.head(15)

,x,l(x),q(x),p(x)
0,15.000000,1000000.000000,0.000040,0.999960
1,15.083333,999959.583333,0.000040,0.999960
2,15.166667,999919.166667,0.000040,0.999960
3,15.250000,999878.750000,0.000040,0.999960
4,15.333333,999838.333333,0.000040,0.999960
5,15.416667,999797.916667,0.000040,0.999960
6,15.500000,999757.500000,0.000040,0.999960
7,15.583333,999717.083333,0.000040,0.999960
8,15.666667,999676.666667,0.000040,0.999960
9,15.750000,999636.250000,0.000040,0.999960


In [78]:
T=tmort_filtered(T_hombre,edad_jubilacion)
T.head(15)

,x,l(x),q(x),p(x)
564,62.000000,897019.000000,0.000766,0.999234
565,62.083333,896331.500000,0.000767,0.999233
566,62.166667,895644.000000,0.000768,0.999232
567,62.250000,894956.500000,0.000768,0.999232
568,62.333333,894269.000000,0.000769,0.999231
569,62.416667,893581.500000,0.000769,0.999231
570,62.500000,892894.000000,0.000770,0.999230
571,62.583333,892206.500000,0.000771,0.999229
572,62.666667,891519.000000,0.000771,0.999229
573,62.750000,890831.500000,0.000772,0.999228


In [81]:
T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)
T

<ipython-input-81-9593b08f4192>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)


,x,l(x),q(x),p(x),n
564,62.000000,897019.000000,0.000766,0.999234,4.007461e-12
565,62.083333,896331.500000,0.000767,0.999233,1.000000e+00
566,62.166667,895644.000000,0.000768,0.999232,2.000000e+00
567,62.250000,894956.500000,0.000768,0.999232,3.000000e+00
568,62.333333,894269.000000,0.000769,0.999231,4.000000e+00
...,...,...,...,...,...
1136,109.666667,121.333333,0.083104,0.916896,5.720000e+02
1137,109.750000,111.250000,0.090637,0.909363,5.730000e+02
1138,109.833333,101.166667,0.099671,0.900329,5.740000e+02
1139,109.916667,91.083333,0.110704,0.889296,5.750000e+02


In [82]:
T['Vn']=np.ones(len(T))*(1/(1+i))
T['Vn']=T.apply(lambda row: row['Vn']**row['n'], axis=1)
T

<ipython-input-82-c643c63f4b06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn']=np.ones(len(T))*(1/(1+i))
<ipython-input-82-c643c63f4b06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn']=T.apply(lambda row: row['Vn']**row['n'], axis=1)


,x,l(x),q(x),p(x),n,Vn
564,62.000000,897019.000000,0.000766,0.999234,4.007461e-12,1.000000
565,62.083333,896331.500000,0.000767,0.999233,1.000000e+00,0.996678
566,62.166667,895644.000000,0.000768,0.999232,2.000000e+00,0.993367
567,62.250000,894956.500000,0.000768,0.999232,3.000000e+00,0.990066
568,62.333333,894269.000000,0.000769,0.999231,4.000000e+00,0.986777
...,...,...,...,...,...,...
1136,109.666667,121.333333,0.083104,0.916896,5.720000e+02,0.149047
1137,109.750000,111.250000,0.090637,0.909363,5.730000e+02,0.148551
1138,109.833333,101.166667,0.099671,0.900329,5.740000e+02,0.148058
1139,109.916667,91.083333,0.110704,0.889296,5.750000e+02,0.147566


In [83]:
sum_vect=T['p(x)']*T['Vn']
suma=sum_vect.sum()
suma

np.float64(254.5545084058316)

In [84]:
c=ahorro_total/suma
c

np.float64(3428739.1080923458)

**EJERCICIO DE MUJER**

In [85]:
#Datos
edad_x=30
sexo='M'
paga=0.04 # el 12% restante lo aporta el empleador
salario = 7000000
edad_jubilacion=57
tasa_anual=0.04

In [86]:
#Calcular el ahorro total de los 57 años

##Tenga en cuenta, el primer pago de la renta es justo a los 30 año, como si fuera hoy
pago_mensual=salario*0.16
n=(edad_jubilacion-edad_x)*12
i=tasa_anual/12
vf=vf_cte(pago_mensual,n,i)
print(vf)

362880000.0
653809052.6404626


In [87]:
ahorro_total=vf
print(f"\033[1mAhorro Total:\033[0m {ahorro_total:.2f}")

Ahorro Total: 653809052.64


In [88]:
T_mujer.head(15)

,x,l(x),q(x),p(x)
0,15.000000,1000000.000000,0.000023,0.999977
1,15.083333,999977.333333,0.000023,0.999977
2,15.166667,999954.666667,0.000023,0.999977
3,15.250000,999932.000000,0.000023,0.999977
4,15.333333,999909.333333,0.000023,0.999977
5,15.416667,999886.666667,0.000023,0.999977
6,15.500000,999864.000000,0.000023,0.999977
7,15.583333,999841.333333,0.000023,0.999977
8,15.666667,999818.666667,0.000023,0.999977
9,15.750000,999796.000000,0.000023,0.999977


In [89]:
T=tmort_filtered(T_mujer,edad_jubilacion)
T

,x,l(x),q(x),p(x)
504,57.000000,959851.000000,0.000277,0.999723
505,57.083333,959585.250000,0.000277,0.999723
506,57.166667,959319.500000,0.000277,0.999723
507,57.250000,959053.750000,0.000277,0.999723
508,57.333333,958788.000000,0.000277,0.999723
...,...,...,...,...
1136,109.666667,295.333333,0.083239,0.916761
1137,109.750000,270.750000,0.090797,0.909203
1138,109.833333,246.166667,0.099865,0.900135
1139,109.916667,221.583333,0.110944,0.889056


In [90]:
T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)
T

<ipython-input-90-9593b08f4192>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)


,x,l(x),q(x),p(x),n
504,57.000000,959851.000000,0.000277,0.999723,3.581135e-12
505,57.083333,959585.250000,0.000277,0.999723,1.000000e+00
506,57.166667,959319.500000,0.000277,0.999723,2.000000e+00
507,57.250000,959053.750000,0.000277,0.999723,3.000000e+00
508,57.333333,958788.000000,0.000277,0.999723,4.000000e+00
...,...,...,...,...,...
1136,109.666667,295.333333,0.083239,0.916761,6.320000e+02
1137,109.750000,270.750000,0.090797,0.909203,6.330000e+02
1138,109.833333,246.166667,0.099865,0.900135,6.340000e+02
1139,109.916667,221.583333,0.110944,0.889056,6.350000e+02


In [91]:
T['Vn']=np.ones(len(T))*(1/(1+i))
T['Vn']=T.apply(lambda row: row['Vn']**row['n'], axis=1)
T

<ipython-input-91-c643c63f4b06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn']=np.ones(len(T))*(1/(1+i))
<ipython-input-91-c643c63f4b06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn']=T.apply(lambda row: row['Vn']**row['n'], axis=1)


,x,l(x),q(x),p(x),n,Vn
504,57.000000,959851.000000,0.000277,0.999723,3.581135e-12,1.000000
505,57.083333,959585.250000,0.000277,0.999723,1.000000e+00,0.996678
506,57.166667,959319.500000,0.000277,0.999723,2.000000e+00,0.993367
507,57.250000,959053.750000,0.000277,0.999723,3.000000e+00,0.990066
508,57.333333,958788.000000,0.000277,0.999723,4.000000e+00,0.986777
...,...,...,...,...,...,...
1136,109.666667,295.333333,0.083239,0.916761,6.320000e+02,0.122070
1137,109.750000,270.750000,0.090797,0.909203,6.330000e+02,0.121664
1138,109.833333,246.166667,0.099865,0.900135,6.340000e+02,0.121260
1139,109.916667,221.583333,0.110944,0.889056,6.350000e+02,0.120857


In [92]:
sum_vect=T['p(x)']*T['Vn']
suma=sum_vect.sum()
suma

np.float64(263.2164186769155)

In [93]:
c=ahorro_total/suma
c

np.float64(2483921.998205512)

**Ahorro inicial $96.000.000**

In [95]:
valor_i=96000000

def vf_cte(pago, n, i, valor_inicial=0):

    pagos = np.ones(n) * pago
    cao_factor = np.ones(n) * (1 + i) ** np.arange(n, 0, -1)
    resp_vector = pagos * cao_factor
    resp = resp_vector.sum() + valor_inicial * (1 + i)**n
    return resp

vf=vf_cte(pago_mensual,n,i,valor_i)
print(vf)

935991032.5096068


In [96]:
ahorro_total=vf
print(f"\033[1mAhorro Total:\033[0m {ahorro_total:.2f}")

Ahorro Total: 935991032.51


In [98]:
T=tmort_filtered(T_mujer,edad_jubilacion)
T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)
T['Vn']=np.ones(len(T))*(1/(1+i))
T['Vn']=T.apply(lambda row: row['Vn']**row['n'], axis=1)

<ipython-input-98-9d274cc27161>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)
<ipython-input-98-9d274cc27161>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn']=np.ones(len(T))*(1/(1+i))
<ipython-input-98-9d274cc27161>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [99]:
sum_vect=T['p(x)']*T['Vn']
suma=sum_vect.sum()
suma

np.float64(263.2164186769155)

In [100]:
c=ahorro_total/suma
c

np.float64(3555975.1067751106)